## Detección de objetos en vídeos con OpenCV DNN 

In [1]:
#Librerias necesarias para el procesamiento de video
import cv2
import numpy as np
import time

**Predicción sobre fotogramas de video**

In [2]:
# Cargar los nombres de las clases COCO
with open('input/object_detection_classes_coco.txt', 'r') as f:
   class_names = f.read().split('\n')
 
# Obtener una matriz de colores diferente para cada una de las clases
COLORS = np.random.uniform(0, 255, size=(len(class_names), 3))
 
# Cargar el modelo DNN
model = cv2.dnn.readNet(model='input/frozen_inference_graph.pb',
                        config='input/ssd_mobilenet_v2_coco_2018_03_29.pbtxt.txt',framework='TensorFlow')
 
# Capturar el vídeo
cap = cv2.VideoCapture('input/video_1.mp4')

# Obtener la anchura y la altura de los fotogramas de vídeo para guardar correctamente los vídeos
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Crear el objeto `VideoWriter()`.
out = cv2.VideoWriter('video_result.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))